# A4: Channel Estimation and Equalization

This notebook simulates a simple OFDM transmission with QPSK modulation, a multipath fading channel, and equalization at the receiver using Least Squares (LS) estimation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, ifft
from scipy.signal import convolve

## Parameters

In [ ]:
# System Parameters
N = 64  # Number of subcarriers
cp_len = 16  # Cyclic prefix length
num_symbols = 10  # Number of OFDM symbols
SNR_dB = 20  # Signal-to-noise ratio

# Channel (example: 3-tap multipath)
channel = np.array([1+0.5j, 0.5-0.3j, 0.2+0.1j])

## Generate QPSK-modulated OFDM symbols

In [ ]:
def qpsk_mod(bits):
    return (2*bits[0::2] - 1) + 1j*(2*bits[1::2] - 1)

# Generate random bits and map to QPSK symbols
bits = np.random.randint(0, 2, N * num_symbols * 2)
qpsk_symbols = qpsk_mod(bits)
symbols = qpsk_symbols.reshape((num_symbols, N))

## OFDM Modulation (IFFT + Cyclic Prefix)

In [ ]:
def add_cp(ofdm_time):
    return np.hstack([ofdm_time[-cp_len:], ofdm_time])

ofdm_tx = []
for i in range(num_symbols):
    time_signal = ifft(symbols[i])
    tx_signal = add_cp(time_signal)
    ofdm_tx.append(tx_signal)

ofdm_tx = np.hstack(ofdm_tx)

## Channel + AWGN

In [ ]:
rx_signal = convolve(ofdm_tx, channel, mode='full')[:len(ofdm_tx)]
noise_power = 10**(-SNR_dB/10)
noise = np.sqrt(noise_power/2) * (np.random.randn(len(rx_signal)) + 1j*np.random.randn(len(rx_signal)))
rx_signal += noise

## Receiver: Remove CP, FFT, Channel Estimation (LS)

In [ ]:
def remove_cp(signal):
    return signal[cp_len:cp_len+N]

rx_symbols = []
H_est = []
for i in range(num_symbols):
    start = i * (N + cp_len)
    segment = rx_signal[start:start + N + cp_len]
    y = fft(remove_cp(segment))
    x = symbols[i]
    h = y / x  # LS estimation (ideal)
    rx_symbols.append(y / h)
    H_est.append(h)

rx_symbols = np.array(rx_symbols).reshape(-1)

## Constellation Plot

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(np.real(rx_symbols), np.imag(rx_symbols), color='blue', alpha=0.5, label='Equalized')
plt.grid(True)
plt.xlabel('In-phase')
plt.ylabel('Quadrature')
plt.title('Constellation of Equalized QPSK Symbols')
plt.legend()
plt.axis('equal')
plt.savefig("modulation_constellation_example.png")
plt.show()